In [9]:
# Load modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
from sksurv.linear_model import CoxnetSurvivalAnalysis
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.exceptions import FitFailedWarning
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, KFold
import seaborn as sns
from Modeling.pen_cox import do_lasso_gcv
from sksurv.preprocessing import OneHotEncoder
import re
from sklearn.preprocessing import LabelBinarizer
from sklearn_pandas import DataFrameMapper 
from sklearn.pipeline import make_pipeline
from source.preprocessing import DataLoader
import xgboost as xgb
from sklearn.model_selection import train_test_split
from source.utils.feature_imp import get_features_cox, get_features_xgb
from Modeling.xgboost import do_XGB_AFT, create_xgb_data



In [2]:
### load data
dl = DataLoader("C:/Users/laeti/OneDrive/Dokumente/LMU/StatisticalConsulting")
dl.load_all_data()

In [3]:
cohorts_exprs_dict = dl.exprs_data
cohorts_pData_dict = dl.pdata_original
intersection_exprs_df = pd.DataFrame(dl.intersection_data['exprs_intersect.csv'])
merged_pData_imputed = pd.DataFrame(dl.merged_pdata_imputed['merged_imputed_pData.csv'])
all_exprs_df =  pd.DataFrame(dl.all_genes_data['all_genes.csv'])
imputed_exprs_df = pd.DataFrame(dl.common_genes_data['common_genes_knn_imputed.csv'])

# 1 Cox-Models

In [10]:
# prepare data for cox
def extract_cohort_name(text):
    regex = r"^([^.]+)\."
    match = re.search(regex, text)
    if match:
        return match.group(1)
    return None

cohort_col = [extract_cohort_name(cor) for cor in merged_pData_imputed.index.tolist()]


merged_pData_imputed
pData_temp = merged_pData_imputed.loc[:, ~merged_pData_imputed.columns.isin(['GLEASON_SCORE_1', 'GLEASON_SCORE_2', 'BCR_STATUS',
                                                                             'MONTH_TO_BCR', 'MONTH_TO_CEP', 'CEP_STATUS'])]
sc_pData = StandardScaler()

#mapper = DataFrameMapper(
#  [(list(set(pData_temp.describe().columns)), StandardScaler())] +
#  [(list(set(pData_temp.columns) - set(pData_temp.describe().columns)), OneHotEncoder())]
#)

pData_temp['TISSUE'] = pData_temp['TISSUE'].astype('category')
pData_temp['CLIN_T_STAGE'] = pData_temp['CLIN_T_STAGE'].astype('category')
pData_temp['PATH_T_STAGE'] = pData_temp['PATH_T_STAGE'].astype('category')
#pData_temp['COHORT'] = cohort_col
#pData_temp['COHORT'] = pData_temp['COHORT'].astype('category')
#X = mapper.fit_transform(pData_temp)

pData_temp_cont = pd.DataFrame(sc_pData.fit_transform(pData_temp[list(set(pData_temp.describe().columns))]), 
                             columns=list(set(pData_temp.describe().columns)), index= pData_temp.index)

pData_temp_cat = OneHotEncoder().fit_transform(pData_temp[(list(set(pData_temp.columns) - set(pData_temp.describe().columns)))])
X = pd.concat([pData_temp_cont, pData_temp_cat, intersection_exprs_df], axis = 1)
#print(pData_temp['COHORT'].info())



y = np.array([(bool(e), t) for e, t in 
              zip(merged_pData_imputed['BCR_STATUS'], merged_pData_imputed['MONTH_TO_BCR'])], dtype = [('event', bool), ('time', float)])




C:\Users\laeti\AppData\Local\Temp\ipykernel_17120\652083402.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pData_temp['TISSUE'] = pData_temp['TISSUE'].astype('category')
C:\Users\laeti\AppData\Local\Temp\ipykernel_17120\652083402.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pData_temp['CLIN_T_STAGE'] = pData_temp['CLIN_T_STAGE'].astype('category')
C:\Users\laeti\AppData\Local\Temp\ipykernel_17120\652083402.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a 

## 1.1 Penalized Cox-Models (integrated feature selection)

### 1.1.1 Pen. Cox model with cohort as covariate (intersection of genes + pData)

In [57]:
gcv_result = do_lasso_gcv('intersect_cohorts', X, y, n_splits = 5, l1_ratio= 1.0, alpha_min_ratio = 0.2)

In [70]:
gcv_result
pd.DataFrame([gcv_result]).to_csv("pen_cox_exprs_pData_wc.csv")

### 1.1.2 Penalized Cox-Model wo. cohort info

In [61]:
gcv_result_woc = do_lasso_gcv('intersect_cohorts', X, y, n_splits = 5, l1_ratio= 1.0, alpha_min_ratio = 0.2)

In [69]:
gcv_result_woc
pd.DataFrame([gcv_result_woc]).to_csv("pen_cox_pData_exprs_woc.csv")

## 1.2 Cox-Model using with pre-selected features


# 2 XGBoost wit AFT

## 2.1 XGBoost with AFT (intersection)

In [4]:
pData_temp = merged_pData_imputed.loc[:, ~merged_pData_imputed.columns.isin(['GLEASON_SCORE_1', 'GLEASON_SCORE_2', 'BCR_STATUS',
                                                                             'MONTH_TO_BCR', 'MONTH_TO_CEP', 'CEP_STATUS'])]

pData_temp['TISSUE'] = pData_temp['TISSUE'].astype('category')
pData_temp['CLIN_T_STAGE'] = pData_temp['CLIN_T_STAGE'].astype('category')
pData_temp['PATH_T_STAGE'] = pData_temp['PATH_T_STAGE'].astype('category')

dtrain, dtest, dvalid, y_train_event, y_test_event, y_valid_event = create_xgb_data(merged_pData_imputed, pData_temp, 'MONTH_TO_BCR', 'BCR_STATUS', intersection_exprs_df)

C:\Users\laeti\AppData\Local\Temp\ipykernel_10236\1117187128.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pData_temp['TISSUE'] = pData_temp['TISSUE'].astype('category')
C:\Users\laeti\AppData\Local\Temp\ipykernel_10236\1117187128.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pData_temp['CLIN_T_STAGE'] = pData_temp['CLIN_T_STAGE'].astype('category')
C:\Users\laeti\AppData\Local\Temp\ipykernel_10236\1117187128.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a 

In [76]:
from itertools import product

max_depths = [3, 6, 10]
lr_rates = [0.05, 0.01]
n_estimators = [500, 1000]
min_child_weight = [5, 10]
la = [1, 100]
alpha = [1, 100]

param_grid = pd.DataFrame(
    list(product(max_depths, lr_rates, n_estimators, min_child_weight, la, alpha)),
    columns=['max_depth', 'lr_rate', 'n_estimators', 'min_child_weight' ,'lambda', 'alpha']
)
param_grid

,max_depth,lr_rate,n_estimators,min_child_weight,lambda,alpha
0,3,0.05,500,5,1,1
1,3,0.05,500,5,1,100
2,3,0.05,500,5,100,1
3,3,0.05,500,5,100,100
4,3,0.05,500,10,1,1
...,...,...,...,...,...,...
91,10,0.01,1000,5,100,100
92,10,0.01,1000,10,1,1
93,10,0.01,1000,10,1,100
94,10,0.01,1000,10,100,1


In [78]:
results_xgb_aft = []
param_grid
for key,row in param_grid.iterrows():
    print(key)
    result = do_XGB_AFT(dtrain, dtest, dvalid, y_train, y_test, y_valid, y_train_event, y_test_event, y_valid_event, row['n_estimators'], 
                        row['lr_rate'], row['max_depth'], row['min_child_weight'], 
                        row['lambda'], row['alpha']) 
    results_xgb_aft.append(result)
    print(result)

0
[0]	train-aft-nloglik:11.73588	eval-aft-nloglik:11.60588
[1]	train-aft-nloglik:10.85469	eval-aft-nloglik:10.75891
[2]	train-aft-nloglik:10.06417	eval-aft-nloglik:10.03857
[3]	train-aft-nloglik:9.34460	eval-aft-nloglik:9.36261
[4]	train-aft-nloglik:8.70346	eval-aft-nloglik:8.72002
[5]	train-aft-nloglik:8.10154	eval-aft-nloglik:8.16490
[6]	train-aft-nloglik:7.56205	eval-aft-nloglik:7.66035
[7]	train-aft-nloglik:7.06692	eval-aft-nloglik:7.21041
[8]	train-aft-nloglik:6.62118	eval-aft-nloglik:6.80095
[9]	train-aft-nloglik:6.21757	eval-aft-nloglik:6.42046
[10]	train-aft-nloglik:5.84487	eval-aft-nloglik:6.06389
[11]	train-aft-nloglik:5.51186	eval-aft-nloglik:5.76519
[12]	train-aft-nloglik:5.20545	eval-aft-nloglik:5.50597
[13]	train-aft-nloglik:4.91623	eval-aft-nloglik:5.23493
[14]	train-aft-nloglik:4.66555	eval-aft-nloglik:5.02187
[15]	train-aft-nloglik:4.42230	eval-aft-nloglik:4.81369
[16]	train-aft-nloglik:4.21120	eval-aft-nloglik:4.63801
[17]	train-aft-nloglik:4.02599	eval-aft-nloglik:4.

In [82]:
results_xgb_aft_df = pd.DataFrame(results_xgb_aft)
results_xgb_aft_df.to_csv('Data/modelling_results_laetti/xgboost_aft/xgb_hp_tuning.csv')
results_xgb_aft_df[results_xgb_aft_df['ci_test'] == results_xgb_aft_df['ci_test'].max()]

### overfitting, best hps: 
#34	0.721135	0.971972	500.0	survival:aft	aft-nloglik	100.0	1.0	0.05	6	hist	True	5.0
#42	0.721135	0.971972	1000.0	survival:aft	aft-nloglik	100.0	1.0	0.05	6	hist	True	5.0

,ci_test,ci_train,num_boost_round,objective,eval_metric,lambda,alpha,learning_rate,max_depth,tree_method,validate_parameters,min_child_weight
34,0.721135,0.971972,500.0,survival:aft,aft-nloglik,100.0,1.0,0.05,6,hist,True,5.0
42,0.721135,0.971972,1000.0,survival:aft,aft-nloglik,100.0,1.0,0.05,6,hist,True,5.0


In [ ]:
## Analyze chose genes: 
path = 'Data/modelling_results_laetti/xgboost_aft/xgb_hp_tuning.csv'
features_xgb_intersect = get_features_xgb(path, 'ci_test', 'importance')

## 2.2 XGBoost with AFT and all genes

In [5]:
pData_temp = merged_pData_imputed.loc[:, ~merged_pData_imputed.columns.isin(['GLEASON_SCORE_1', 'GLEASON_SCORE_2', 'BCR_STATUS',
                                                                             'MONTH_TO_BCR', 'MONTH_TO_CEP', 'CEP_STATUS'])]

pData_temp['TISSUE'] = pData_temp['TISSUE'].astype('category')
pData_temp['CLIN_T_STAGE'] = pData_temp['CLIN_T_STAGE'].astype('category')
pData_temp['PATH_T_STAGE'] = pData_temp['PATH_T_STAGE'].astype('category')

dtrain, dtest, dvalid, y_train_event, y_test_event, y_valid_event = create_xgb_data(merged_pData_imputed, pData_temp, 'MONTH_TO_BCR', 'BCR_STATUS', all_exprs_df)

C:\Users\laeti\AppData\Local\Temp\ipykernel_10236\3452433967.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pData_temp['TISSUE'] = pData_temp['TISSUE'].astype('category')
C:\Users\laeti\AppData\Local\Temp\ipykernel_10236\3452433967.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pData_temp['CLIN_T_STAGE'] = pData_temp['CLIN_T_STAGE'].astype('category')
C:\Users\laeti\AppData\Local\Temp\ipykernel_10236\3452433967.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a 

In [7]:
from itertools import product

max_depths = [3]
lr_rates = [0.1]
n_estimators = [250]
min_child_weight = [5]
la = [100]
alpha = [1]

param_grid = pd.DataFrame(
    list(product(max_depths, lr_rates, n_estimators, min_child_weight, la, alpha)),
    columns=['max_depth', 'lr_rate', 'n_estimators', 'min_child_weight' ,'lambda', 'alpha']
)
param_grid

,max_depth,lr_rate,n_estimators,min_child_weight,lambda,alpha
0,3,0.1,250,5,100,1


In [8]:
results_xgb_aft_all = []
param_grid
for key,row in param_grid.iterrows():
    print(key)
    result = do_XGB_AFT(dtrain, dtest, dvalid, dtrain.get_label(), dtest.get_label(), dvalid.get_label(), y_train_event, y_test_event, y_valid_event, row['n_estimators'], 
                        row['lr_rate'], row['max_depth'], row['min_child_weight'], 
                        row['lambda'], row['alpha']) 
    results_xgb_aft_all.append(result)
    print(result)

0
[0]	train-aft-nloglik:11.05049	eval-aft-nloglik:10.92195
[1]	train-aft-nloglik:9.66467	eval-aft-nloglik:9.63032
[2]	train-aft-nloglik:8.49376	eval-aft-nloglik:8.46891
[3]	train-aft-nloglik:7.51230	eval-aft-nloglik:7.56198
[4]	train-aft-nloglik:6.68080	eval-aft-nloglik:6.75083
[5]	train-aft-nloglik:5.98039	eval-aft-nloglik:6.09078
[6]	train-aft-nloglik:5.38703	eval-aft-nloglik:5.53106
[7]	train-aft-nloglik:4.88333	eval-aft-nloglik:5.04285
[8]	train-aft-nloglik:4.44445	eval-aft-nloglik:4.62780
[9]	train-aft-nloglik:4.08380	eval-aft-nloglik:4.28681
[10]	train-aft-nloglik:3.77048	eval-aft-nloglik:3.97512
[11]	train-aft-nloglik:3.50717	eval-aft-nloglik:3.73788
[12]	train-aft-nloglik:3.27438	eval-aft-nloglik:3.52499
[13]	train-aft-nloglik:3.06704	eval-aft-nloglik:3.34171
[14]	train-aft-nloglik:2.89455	eval-aft-nloglik:3.18204
[15]	train-aft-nloglik:2.74838	eval-aft-nloglik:3.04955
[16]	train-aft-nloglik:2.61975	eval-aft-nloglik:2.93121
[17]	train-aft-nloglik:2.50824	eval-aft-nloglik:2.8336

In [14]:
# lool into feature importance: 
results_xgb_aft_all_df = pd.DataFrame(results_xgb_aft_all)
importance = results_xgb_aft_all_df['importance'][0]
importance_df = pd.DataFrame({'col':importance.keys(), 'gain':importance.values()}).sort_values(by = 'gain', ascending=False)
print(importance_df.col.values)
importance_df

#features_xgb_all = get_features_xgb(path, 'ci_test', 'importance')

['ENSG00000283047' 'ENSG00000225401' 'ENSG00000286883' 'ENSG00000254788'
 'ENSG00000238072' 'ENSG00000273420' 'ENSG00000268316' 'ENSG00000267452'
 'ENSG00000279561' 'ENSG00000273139' 'ENSG00000232653' 'ENSG00000279901'
 'ENSG00000215009' 'ENSG00000131152' 'ENSG00000254801' 'ENSG00000279237'
 'ENSG00000273341' 'ENSG00000272196' 'ENSG00000275572' 'ENSG00000279700'
 'ENSG00000198156' 'ENSG00000275234' 'ENSG00000270751' 'ENSG00000124208'
 'ENSG00000278897' 'ENSG00000241983' 'ENSG00000174370' 'PATH_T_STAGE'
 'ENSG00000204510' 'ENSG00000254433' 'ENSG00000169203' 'ENSG00000155659'
 'ENSG00000279406' 'ENSG00000244056' 'ENSG00000230325' 'ENSG00000176900'
 'ENSG00000266472' 'ENSG00000250892' 'ENSG00000083812' 'ENSG00000187627'
 'ENSG00000086696' 'ENSG00000102317' 'ENSG00000084754' 'ENSG00000136244'
 'ENSG00000146038' 'ENSG00000244671' 'ENSG00000124657' 'ENSG00000007933'
 'ENSG00000203811' 'ENSG00000283706' 'ENSG00000223953' 'ENSG00000112799'
 'ENSG00000164292' 'ENSG00000233822' 'ENSG00000085514'

,col,gain
219,ENSG00000283047,445.246094
160,ENSG00000225401,409.874512
220,ENSG00000286883,322.920410
186,ENSG00000254788,310.213989
174,ENSG00000238072,298.966553
...,...,...
197,ENSG00000264522,0.700005
156,ENSG00000213588,0.484547
176,ENSG00000243364,0.413214
81,ENSG00000141644,0.397619


In [11]:
cox_lasso_all_cohorts = pd.read_csv('./Data/feature_sel/pen_cox/feat_sel_pen_lasso_per_cohort.csv')
print(len(cox_lasso_all_cohorts))
genes_list_cohorts = []
for index, row in cox_lasso_all_cohorts.iterrows():
    genes = row[['non_zero_genes']][0]
    gene_list = eval(genes)
    #print(len(gene_list))
    genes_list_cohorts.append(gene_list)

genes_list_chained_cohorts = [elem for sublist in genes_list_cohorts for elem in sublist]
#print(len(genes_list_chained_cohorts))
#counts_cohorts = Counter(genes_list_chained_cohorts)
#print(counts_cohorts)
#print(len(list(set(genes_list_chained_cohorts))))


cox_lasso_all_intersect = pd.read_csv('./Data/feature_sel/pen_cox/feat_sel_pen_lasso_intersect_cohort.csv')
print(len(cox_lasso_all_intersect))

genes_list_intersect= eval(cox_lasso_all_intersect['non_zero_genes'].iloc[0])

cox_lasso_all_imputed = pd.read_csv('Data/feature_sel/pen_cox/feat_sel_pen_lasso_imputed.csv')
genes_list_imputed= eval(cox_lasso_all_imputed['non_zero_genes'].iloc[0])



print(len(list(set(genes_list_imputed) & set(genes_list_chained_cohorts))))
print(len(list(set(genes_list_intersect) & set(genes_list_chained_cohorts))))
print(len(list(set(genes_list_imputed) & set(genes_list_intersect))))

9
1
8
7
48


C:\Users\laeti\AppData\Local\Temp\ipykernel_10236\3256741250.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  genes = row[['non_zero_genes']][0]
C:\Users\laeti\AppData\Local\Temp\ipykernel_10236\3256741250.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  genes = row[['non_zero_genes']][0]
C:\Users\laeti\AppData\Local\Temp\ipykernel_10236\3256741250.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  genes = row[['non_zero_gene

In [18]:
print(len(list(set(importance_df.col.values) & set(genes_list_chained_cohorts))))

print(len(list(set(importance_df.col.values) & set(genes_list_intersect))))
print(len(list(set(importance_df.col.values) & set(genes_list_imputed))))

7
9
9


# 3 CatBoost with CPH-assumption

In [66]:
# prepare data for cat boost 
pData_temp = merged_pData_imputed.loc[:, ~merged_pData_imputed.columns.isin(['GLEASON_SCORE_1', 'GLEASON_SCORE_2', 'MONTH_TO_CEP', 'CEP_STATUS'])]
pData_temp['TISSUE'] = pData_temp['TISSUE'].astype('category')
pData_temp['CLIN_T_STAGE'] = pData_temp['CLIN_T_STAGE'].astype('category')
pData_temp['PATH_T_STAGE'] = pData_temp['PATH_T_STAGE'].astype('category')

X = pd.concat([pData_temp, imputed_exprs_df], axis = 1)
X['label'] = np.where(pData_temp['BCR_STATUS'] == 1, pData_temp['MONTH_TO_BCR'], - pData_temp['MONTH_TO_BCR'])
features = list(X.columns)
features.remove('MONTH_TO_BCR')
features.remove('label')
features.remove('BCR_STATUS')
cat_features = list(set(X.columns) - set(X.describe().columns))

X_train, X_test = train_test_split(X, test_size=0.2, random_state=42)


C:\Users\laeti\AppData\Local\Temp\ipykernel_10236\469677867.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pData_temp['TISSUE'] = pData_temp['TISSUE'].astype('category')
C:\Users\laeti\AppData\Local\Temp\ipykernel_10236\469677867.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pData_temp['CLIN_T_STAGE'] = pData_temp['CLIN_T_STAGE'].astype('category')
C:\Users\laeti\AppData\Local\Temp\ipykernel_10236\469677867.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Dat

In [67]:
from catboost import CatBoostRegressor, Pool

train_pool = Pool(X_train[features], label=X_train['label'], cat_features=cat_features)
test_pool = Pool(X_test[features], label=X_test['label'], cat_features=cat_features)

model = CatBoostRegressor(iterations=100,
                          loss_function='Cox',
                          eval_metric='Cox',
                          metric_period=50
                         )

In [68]:
model.fit(train_pool, eval_set=test_pool)

0:	learn: -1498.3825885	test: -255.6245195	best: -255.6245195 (0)	total: 3.89s	remaining: 6m 24s
50:	learn: -1440.0575084	test: -251.5023343	best: -251.5023343 (50)	total: 1m 39s	remaining: 1m 35s
99:	learn: -1394.9059743	test: -249.1300426	best: -249.1300426 (99)	total: 3m 7s	remaining: 0us

bestTest = -249.1300426
bestIteration = 99



In [63]:
from lifelines.utils import concordance_index

train_y_pred = model.predict(train_pool)
test_y_pred = model.predict(test_pool)

ci_train = concordance_index(X_train['MONTH_TO_BCR'], -train_y_pred, X_train['BCR_STATUS'])
ci_test = concordance_index(X_test['MONTH_TO_BCR'], -test_y_pred, X_test['BCR_STATUS'])
print(f'Train set. Concordance index:{ci_train:0.5f}')
print(f'Test set. Concordance index:{ci_test:0.5f}')

CatBoostError: There is no trained model to use predict(). Use fit() to train model. Then use this method.